In [55]:
class DefaultAttributes:
    def __init__(self):
        self.attributes = {}
        
    def __getattr__(self, name):
        # 존재하지 않는 속성에 대한 기본 값을 반환하거나, 로그를 기록할 수 있습니다.
        return self.attributes.get(name, f"{name} 속성은 존재하지 않습니다.")
    
    def __setattr__(self, name, value):
        if name == "attributes":
            print(f"__setattr__ called : {name} = {value}")
            super().__setattr__(name, value)
        else:
            print(f"Setting Value : {name} = {value}")
            self.attributes[name] = value

In [56]:
obj = DefaultAttributes()

__setattr__ called : attributes = {}


In [57]:
obj.__dict__

{'attributes': {}}

In [58]:
obj.color

'color 속성은 존재하지 않습니다.'

In [59]:
obj.__dict__

{'attributes': {}}

In [60]:
obj.color = 'red'
obj.__dict__

Setting Value : color = red


{'attributes': {'color': 'red'}}

In [83]:
class Proxy:
    def __init__(self, obj):
        self._obj = obj
    
    # 자신에게 없는 속성을 접근했을 때, 호출됨.    
    def __getattr__(self, name):
        # 동적으로 속성을 생성하거나 기본 값을 제공할 수 있음.
        # 속성 접근 시 자동으로 처리할 수 있는 추가 로직을 삽입 기능 (예: 로그 기록, 기본값 설정)
        print(f"Proxy.__getattr__(name={name})")
        return getattr(self._obj, name)
    
    # 자신에게 있던 없든 관계없이 무조건 호출됨.
    # 만약, 이름이 밑줄"_"로 시작하면, super()를 사용해서 __setter__() 원래의 구현을 호출한다.
    def __setattr__(self, name, value):
        if name.startswith("_"):
            print(f"Proxy.__setattr__(name={name}, value={value})")
            # __setter__()구현에서 이름 확인이 들어 있다. 만약 이름이 밑줄로 시작하면 super()를 사용해서 __setter__()의 원래 구현을 호출한다.
            super().__setattr__(name, value) 
            # self._obj = obj 호출 시 부모의 __setattr__ 함수 호출 필요!
            # 특별 메소드를 오버라이드 한 코드에서 super()를 사용하기도 한다.
        else:
            print(f"Setting Object {name} = {value}")
            setattr(self._obj, name, value)
         


In [84]:
class A:
    def __init__(self, x):
        self.x = x
    def spam(self):
        print("A.spam")
        return 1
    

In [85]:
a = A(42)

In [86]:
a.__dict__

{'x': 42}

In [87]:
getattr(a, 'x')

42

In [88]:
p = Proxy(a)

Proxy.__setattr__(name=_obj, value=<__main__.A object at 0x1073cf770>)


In [89]:
p._obj

In [90]:
spam_value = p.spam()
spam_value

Proxy.__getattr__(name=spam)
A.spam


1

In [91]:
p.x = 42

setting x = 42


In [92]:
p.x

Proxy.__getattr__(name=x)


42

In [93]:
p.__dict__

{'_obj': <__main__.A at 0x1073cf770>}

In [95]:
p.y=10

setting y = 10


In [96]:
p.y

Proxy.__getattr__(name=y)


10

In [97]:
p.__dict__

{'_obj': <__main__.A at 0x1073cf770>}

In [98]:
a.__dict__

{'x': 42, 'y': 10}

In [99]:
p._obj = 'test'

Proxy.__setattr__(name=_obj, value=test)


In [100]:
p.__dict__

{'_obj': 'test'}

In [15]:
""" 
새로운 클로스나, 인스턴스 속성 만들기.
    - 새로운 종류의 인스턴스 속성을 만드려면, 그 기능을 디스크립터 클래스 형태로 정의
"""

class Integer:
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, cls):
        # print("__get__ Method Called")
        if instance is None:
            print("instance is None!")
            return self
        else:
            # print(f"instance: {instance}, class: {cls}")
            # print(f"get : {self.name}")
            return instance.__dict__[self.name]
        
    def __set__(self, instance, value):
        # print("__set__ Method Called")
        if not isinstance(value, int):
            # 타입 확인 추가적 기능 추가
            raise TypeError("Expected an int")
        # print(f"instance: {instance}")
        # print(f"set : {self.name} = {value}")
        instance.__dict__[self.name] = value
        
    def __delete__(self, instance):
        del instance.__dict__[self.name]

class Point:
    x = Integer('x')
    y = Integer('y')
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        print(id(x) == id(self.x))

In [16]:
p = Point(2, 3)

True
